# Preprocessing for Model Development as Described in "Risk Stratification with Explainable Machine Learning for 30-Day Procedure-Related Mortality and 30-Day Unplanned Readmission in Patients with Peripheral Arterial Disease"

Meredith Cox; J.C. Panagides; Azadeh Tabari, MD; Sanjeeva Kalva, MD; Jayashree Kalpathy-Cramer, PhD; Dania Daye, MD, PhD


## Imports

In [1]:
import copy
import numpy as np
import pandas as pd
import pymrmr

from imblearn.over_sampling import SMOTE, ADASYN
from interpretableai import iai
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split

# NSQIP Preprocessing

In [ ]:
# Extracting NSQIP data for lower extremity endovascular interventions
# Files are available only upon request: https://www.facs.org/quality-programs/acs-nsqip/participant-use/puf-form

puf_2018 = pd.read_csv("./2018/acs_nsqip_puf18.txt",delimiter='\t')
puf_2017 = pd.read_csv("./2017/acs_nsqip_puf17.txt",delimiter='\t')
puf_2016 = pd.read_csv("./2016/acs_nsqip_puf16.txt",delimiter='\t')
puf_2015 = pd.read_csv("./2015/acs_nsqip_puf15_v2.txt",delimiter='\t')
puf_2014 = pd.read_csv("./2014/acs_nsqip_puf14.txt",delimiter='\t')
puf_2013 = pd.read_csv("./2013/acs_nsqip_puf13.txt",delimiter='\t')
puf_2012 = pd.read_csv("./2012/acs_nsqip_puf12.txt",delimiter='\t')
puf_2011 = pd.read_csv("./2011/ACS_NSQIP_PUF11_TXT.txt",delimiter='\t')
puf_2010 = pd.read_csv("./2010/ACS_NSQIP_PUF10.txt",delimiter='\t')
puf_2009 = pd.read_csv("./2009/ACS_NSQIP_PUF09_txt.txt",delimiter='\t')
puf_2008 = pd.read_csv("./2008/ACS_NSQIP_PUF08.txt",delimiter='\t')

puf_2018.columns = puf_2018.columns.str.upper()
puf_2017.columns = puf_2017.columns.str.upper()
puf_2016.columns = puf_2016.columns.str.upper()
puf_2015.columns = puf_2015.columns.str.upper()
puf_2014.columns = puf_2014.columns.str.upper()
puf_2013.columns = puf_2013.columns.str.upper()
puf_2012.columns = puf_2012.columns.str.upper()
puf_2011.columns = puf_2011.columns.str.upper()
puf_2010.columns = puf_2010.columns.str.upper()
puf_2009.columns = puf_2009.columns.str.upper()
puf_2008.columns = puf_2008.columns.str.upper()

all_puf = pd.concat([puf_2008,
                     puf_2009,
                     puf_2010,
                     puf_2011,
                     puf_2012,
                     puf_2013,
                     puf_2014,
                     puf_2015,
                     puf_2016,
                     puf_2017,
                     puf_2018], 
                    ignore_index=True)
all_puf['CPT'] = all_puf.CPT.astype(str)

le_endo_cpt = ["37224","37225","37226","37227","37228","37229","37230","37231","37232","37233","37234"]
le_endo_cases = all_puf[all_puf.CPT.isin(le_endo_cpt)]
le_endo_cases.dropna(thresh=1, axis=1) # drop columns with all NA values
le_endo_cases.to_csv("NSQIP_LEE_2008-2018.csv",index=False) # write to csv for eaasier reading later

In [ ]:
# Merge targeted vacular module
# Files are also available only upon request

lee_2018 = pd.read_csv("./2018_Procedure_Targeted/puf_tar_lee_2018.txt",delimiter="\t")
lee_2017 = pd.read_csv("./2017_Procedure_targeted/puf_tar_lee_2017.txt",delimiter="\t")
lee_2016 = pd.read_csv("./2016_Procedure_Targeted/puf_tar_lee_2016.txt",delimiter="\t")
lee_2015 = pd.read_csv("./2015_Procedure_Targeted/puf_tar_lee_2015.txt",delimiter="\t")
lee_2014 = pd.read_csv("./2014_Procedure_Targeted/puf_tar_lee_2014.txt",delimiter="\t")
lee_2013 = pd.read_csv("./2013_Procedure_Targeted/puf_tar_lee_2013.txt",delimiter="\t")
lee_2012 = pd.read_csv("./2012_Procedure_Targeted/puf_tar_lee_2012.txt",delimiter="\t")
lee_2011 = pd.read_csv("./2011_Procedure_Targeted/puf_tar_lee_2011.txt",delimiter="\t")

all_lee = pd.concat([lee_2011,lee_2012,lee_2013,lee_2014,lee_2015,lee_2016,lee_2017,lee_2018], ignore_index=True)
lee_merged = all_lee.merge(le_endo_cases, on='CASEID', how='left')
lee_merged = lee_merged.dropna(thresh=1, axis=1) # drop columns with all NA values
lee_merged.to_csv("NSQIP_LEE_2008-2018_merged.csv",index=False)

In [ ]:
# Remove non-useful columns

lee_merged_filtered = lee_merged.drop(columns=['CASEID', 'LEE_DULP','LEE_DBLEEDING','LEE_DMI_STROKE',
                                               'LEE_DWOUND','LEE_DMOSTSEVOUTCOME','PRNCPTX','WORKRVU',
                                               'INOUT','TRANST','ADMYR','ADMSYR','OPERYR','ATTEND',
                                               'SURGSPEC','DNR','DPRNA','DPRBUN','DPRCREAT','DPRALBUM',
                                               'DPRBILI','DPRSGOT','DPRALKPH','DPRWBC','DPRHCT','DPRPLATE',
                                               'DPRPTT','DPRPT','DPRINR','OTHERWRVU1','OTHERWRVU2','OTHERWRVU3',
                                               'OTHERWRVU4','OTHERWRVU5','OTHERWRVU6','OTHERWRVU7','OTHERWRVU8',
                                               'OTHERWRVU9','OTHERWRVU10','CONWRVU1','CONWRVU2','CONWRVU3',
                                               'CONWRVU4','CONWRVU5','CONWRVU6','CONWRVU7','CONWRVU8','CONWRVU9',
                                               'CONWRVU10', 'PGY','ANESURG','SURGANE','DPATRM','ANETIME','SDISDT',
                                               'HDISDT','YRDEATH','TOTHLOS','DSUPINFEC','DWNDINFD','DORGSPCSSI',
                                               'DDEHIS','DOUPNEUMO','DREINTUB','DPULEMBOL','DFAILWEAN',
                                               'DRENAINSF','DOPRENAFL','DURNINFEC','DCNSCVA','DCNSCOMA',
                                               'DNEURODEF','DCDARREST','DCDMI','DOTHBLEED','DOTHGRAFL',
                                               'DOTHDVT','DOTHSYSEP','DOTHSESHOCK','PODIAGTX','DSDTOHD','DOPTODIS',
                                               'MORTPROB','MORBPROB','DISCHDEST','RETOR2PODAYS','READMPODAYS2',
                                               'READMPODAYS3','READMPODAYS4','READMPODAYS5','DOTHCDIFF',
                                               'LEE_DMOSTSEVOUTC','ADMQTR','ANESTHES','OTHERPROC1','OTHERCPT1',
                                               'OTHERPROC2','OTHERCPT2','OTHERPROC3','OTHERCPT3','OTHERPROC4',
                                               'OTHERCPT4','OTHERPROC5','OTHERCPT5','OTHERPROC6','OTHERCPT6',
                                               'OTHERPROC7','OTHERCPT7','OTHERPROC8','OTHERCPT8','OTHERPROC9',
                                               'OTHERCPT9','OTHERPROC10','OTHERCPT10','CONCURR1','CONCPT1',
                                               'CONCURR2','CONCPT2','CONCURR3','CONCPT3','CONCURR4','CONCPT4',
                                               'CONCURR10','MALLAMP','RBC','OPTIME','HTOODAY','STOODAY','TOTSLOS',
                                               'NSUPINFEC','NWNDINFD','NORGSPCSSI','NDEHIS','NOUPNEUMO','NREINTUB',
                                               'NPULEMBOL','NFAILWEAN','NRENAINSF','NOPRENAFL','NURNINFEC','NCNSCVA',
                                               'NCNSCOMA','CNSCOMA','NNEURODEF','NEURODEF','NCDARREST',
                                               'NCDMI','NOTHBLEED','NOTHGRAFL','OTHGRAFL','OTHGRAFL','NOTHDVT',
                                               'NOTHSYSEP','NOTHSESHOCK','PODIAG','RETURNOR','READMISSION',
                                               'STILLINHOSP','REOPORCPT1','RETORRELATED','REOPORICD91',
                                               'REOPERATION2','REOPOR2CPT1','RETOR2RELATED','REOPOR2ICD91',
                                               'REOPERATION3','READMISSION1','READMRELATED1','READMSUSPREASON1',
                                               'READMRELICD91','READMISSION2','UNPLANNEDREADMISSION2',
                                               'READMRELATED2','READMSUSPREASON2','READMRELICD92','READMRELICD92',
                                               'READMISSION3','UNPLANNEDREADMISSION3','READMRELATED3',
                                               'READMISSION4','READMISSION5','READMUNRELSUSP1','READMUNRELICD91',
                                               'READMUNRELSUSP2','READMUNRELICD92','READMUNRELSUSP3',
                                               'READMUNRELICD93','PODIAG10','PODIAGTX10','REOPOR1ICD101',
                                               'REOPOR2ICD101','READMRELICD101','READMUNRELICD101','READMRELICD102',
                                               'READMUNRELICD102','READMRELICD103','WOUND_CLOSURE','PODIAG_OTHER',
                                               'PODIAG_OTHER10','ANESTHES_OTHER','NOTHCDIFF','READMSUSPREASON3',
                                               'SSSIPATOS','DSSIPATOS','OSSIPATOS','PNAPATOS','UTIPATOS',
                                               'VENTPATOS','SEPSISPATOS','SEPSHOCKPATOS','LEE_PREHEMO',
                                               'LEE_POSTHEMO','OTHCDIFF','CPT','LEE_HRF_ANAT','LEE_PROC','PACKS', 
                                               'ETOH', 'CPNEUMON', 'ESOVAR', 'HXMI', 'PRVPCI', 'PRVPCS','HXANGINA', 
                                               'HXPVD', 'RESTPAIN', 'IMPSENS', 'COMA', 'HEMI', 'HXTIA','CVA', 
                                               'CVANO', 'TUMORCNS', 'PARA', 'QUAD','CHEMO','RADIO','PREGNANCY', 
                                               'PROPER30'
                                              ])

lee_merged_filtered["Symptoms_Critical_limb_ischemia_tissue_loss"] = (lee_merged_filtered.LEE_SYMPT == 'Critical limb ischemia: tissue loss').map({True: 1, False: 0}) 
lee_merged_filtered['Symptoms_Claudication'] = (lee_merged_filtered.LEE_SYMPT == 'Claudication').map({True: 1, False: 0}) 
lee_merged_filtered['Symptoms_Critical_limb_ischemia_rest_pain'] = (lee_merged_filtered.LEE_SYMPT == 'Critical limb ischemia: rest pain').map({True: 1, False: 0}) 
lee_merged_filtered['Symptoms_Asymptomatic'] = (lee_merged_filtered.LEE_SYMPT == 'Asymptomatic').map({True: 1, False: 0}) 
lee_merged_filtered['Symptoms_Not_documented'] = (lee_merged_filtered.LEE_SYMPT == 'Not documented').map({True: 1, False: 0}) 

lee_merged_filtered['MRTAS'] = (lee_merged_filtered.LEE_MRTAS == 'Yes').map({True: 1, False: 0}) 
lee_merged_filtered['Race_american_indian_alaska_native'] = (lee_merged_filtered.RACE_NEW== 'American Indian or Alaska Native').map({True: 1, False: 0})
lee_merged_filtered['Race_black'] = (lee_merged_filtered.RACE_NEW== 'Black or African American').map({True: 1, False: 0})
lee_merged_filtered['Race_native_hawaiian_pacific_islander'] = (lee_merged_filtered.RACE_NEW== 'Native Hawaiian or Pacific Islander').map({True: 1, False: 0})
lee_merged_filtered['Race_asian'] = (lee_merged_filtered.RACE_NEW== 'Asian').map({True: 1, False: 0})
lee_merged_filtered['Race_unknown'] = (lee_merged_filtered.RACE_NEW== 'Unknown/Not Reported').map({True: 1, False: 0})
lee_merged_filtered['Race_white'] = (lee_merged_filtered.RACE_NEW== 'White').map({True: 1, False: 0})
lee_merged_filtered['bmi'] =  703 * (lee_merged_filtered.WEIGHT / (lee_merged_filtered.HEIGHT **2))
lee_merged_filtered['obese'] =  (lee_merged_filtered.bmi >= 30).map({True: 1, False: 0})
lee_merged_filtered['diabetes'] =  (lee_merged_filtered.DIABETES== 'NO').map({True: 0, False: 1})
lee_merged_filtered['smoke'] =  (lee_merged_filtered.SMOKE== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['dyspnea'] =  (lee_merged_filtered.DYSPNEA== 'No').map({True: 0, False: 1})
lee_merged_filtered['fnstatus'] =  lee_merged_filtered.FNSTATUS2.map({'Independent': 0, 'Partially Dependent': 1,'Totally Dependent':1})

lee_merged_filtered.loc[(lee_merged_filtered.VENTILAT == 'Yes') | (lee_merged_filtered.HXCOPD == 'Yes'), 'pulmcomorb'] = 1
lee_merged_filtered.loc[(lee_merged_filtered.VENTILAT == 'No') & (lee_merged_filtered.HXCOPD == 'No'), 'pulmcomorb'] = 0
    
lee_merged_filtered.loc[(lee_merged_filtered.ASCITES == 'Yes') | (lee_merged_filtered.HXCHF == 'Yes') | (lee_merged_filtered.HYPERMED == 'Yes'), 'cardiaccomorb'] = 1
lee_merged_filtered.loc[(lee_merged_filtered.ASCITES == 'No') & (lee_merged_filtered.HXCHF == 'No') & (lee_merged_filtered.HYPERMED == 'No'), 'cardiaccomorb'] = 0

lee_merged_filtered.loc[(lee_merged_filtered.RENAFAIL == 'Yes') | (lee_merged_filtered.DIALYSIS == 'Yes'), 'renalcomorb'] = 1
lee_merged_filtered.loc[(lee_merged_filtered.RENAFAIL == 'No') & (lee_merged_filtered.DIALYSIS == 'No'), 'renalcomorb'] = 0

lee_merged_filtered['wndinf'] =  (lee_merged_filtered.WNDINF== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['steroid'] =  (lee_merged_filtered.STEROID== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['wtloss'] =  (lee_merged_filtered.WTLOSS== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['bleeddis'] =  (lee_merged_filtered.BLEEDDIS== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['transfus'] =  (lee_merged_filtered.TRANSFUS== 'Yes').map({True: 1, False: 0})


lee_merged_filtered['WND3'] =  (lee_merged_filtered.WNDCLAS).map({'1-Clean': 0, '2-Clean/Contaminated': 0,
                                                                 '3-Contaminated':1,'4-Dirty/Infected':1})
lee_merged_filtered['ASA3'] =  (lee_merged_filtered.ASACLAS).map({'3-Severe Disturb':1, '4-Life Threat':1,'2-Mild Disturb':0, 
                                                                  '5-Moribund':1, '1-No Disturb':0})

lee_merged_filtered['LEE_HRF_PHYS'] =  (lee_merged_filtered.LEE_HRF_PHYS).map({'Yes':1, 'No':0})
lee_merged_filtered['LEE_PREMED_ASPIRIN'] =  (lee_merged_filtered.LEE_PREMED_ASPIRIN).map({'Yes':1, 'No':0})
lee_merged_filtered['LEE_PREMED_STATIN'] =  (lee_merged_filtered.LEE_PREMED_STATIN).map({'Yes':1, 'No':0})
lee_merged_filtered['LEE_PREMED_BETAB'] =  (lee_merged_filtered.LEE_PREMED_BETAB).map({'Yes':1, 'No':0})
lee_merged_filtered['LEE_WOUND'] =  (lee_merged_filtered.LEE_WOUND).map({'Yes':1, 'No':0})
lee_merged_filtered['LEE_AMPUTATION'] =  (lee_merged_filtered.LEE_AMPUTATION).map({'Yes':1, 'No':0})
lee_merged_filtered['SEX'] =  (lee_merged_filtered.SEX).map({'female':0, 'male':1})
lee_merged_filtered['DISCANCR'] =  (lee_merged_filtered.DISCANCR).map({'Yes':1, 'No':0})
lee_merged_filtered['EMERGNCY'] =  (lee_merged_filtered.EMERGNCY).map({'Yes':1, 'No':0})
lee_merged_filtered['ELECTSURG'] =  (lee_merged_filtered.ELECTSURG).map({'Yes':1, 'No':0})

lee_merged_filtered['LEE_ULP'] =  (lee_merged_filtered.LEE_ULP== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['LEE_BLEEDING'] =  (lee_merged_filtered.LEE_BLEEDING== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['LEE_MI_STROKE'] =  (lee_merged_filtered.LEE_MI_STROKE== 'Yes').map({True: 1, False: 0})
lee_merged_filtered['SUPINFEC'] =  (lee_merged_filtered.SUPINFEC== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['WNDINFD'] =  (lee_merged_filtered.WNDINFD== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['ORGSPCSSI'] =  (lee_merged_filtered.ORGSPCSSI== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['DEHIS'] =  (lee_merged_filtered.DEHIS== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['OUPNEUMO'] =  (lee_merged_filtered.OUPNEUMO== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['REINTUB'] =  (lee_merged_filtered.REINTUB== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['PULEMBOL'] =  (lee_merged_filtered.PULEMBOL== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['FAILWEAN'] =  (lee_merged_filtered.FAILWEAN== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['RENAINSF'] =  (lee_merged_filtered.RENAINSF== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['OPRENAFL'] =  (lee_merged_filtered.OPRENAFL== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['URNINFEC'] =  (lee_merged_filtered.URNINFEC== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['CNSCVA'] =  (lee_merged_filtered.CNSCVA== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['CDARREST'] =  (lee_merged_filtered.CDARREST== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['CDMI'] =  (lee_merged_filtered.CDMI== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['OTHBLEED'] =  (lee_merged_filtered.OTHBLEED== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['OTHDVT'] =  (lee_merged_filtered.OTHDVT== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['OTHSYSEP'] =  (lee_merged_filtered.OTHSYSEP== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['OTHSESHOCK'] =  (lee_merged_filtered.OTHSESHOCK== 'No Complication').map({True: 0, False: 1})
lee_merged_filtered['PRSEPIS'] = (lee_merged_filtered.PRSEPIS== 'None').map({True: 0, False: 1})

lee_merged_filtered['DEATH'] =  (lee_merged_filtered.LEE_MOSTSEVOUTCOME).map({'Death': 1,'Not documented': None,
                                                                             'Patent treated arterial segment, no stenosis':0,
                                                                             'Patent treated arterial segment with stenosis':0,
                                                                             'Reintervened treated arterial segment with stenosis':0,
                                                                             'Major Amputation':0, 'Image-proven treated arterial segment thrombosis or clinically evident thrombosis with no planned intervention':0,
                                                                             'Other':0,'New bypass in the treated arterial segment':0,
                                                                             'Clinically Patent Graft':0,'Reintervened treated arterial segment with no current stenosis':0})

lee_merged_filtered['THROMBOSIS'] =  (lee_merged_filtered.LEE_MOSTSEVOUTCOME).map({'Death': 0,'Not documented': None,
                                                                             'Patent treated arterial segment, no stenosis':0,
                                                                             'Patent treated arterial segment with stenosis':0,
                                                                             'Reintervened treated arterial segment with stenosis':0,
                                                                             'Major Amputation':0, 'Image-proven treated arterial segment thrombosis or clinically evident thrombosis with no planned intervention':1,
                                                                             'Other':0,'New bypass in the treated arterial segment':0,
                                                                             'Clinically Patent Graft':0,'Reintervened treated arterial segment with no current stenosis':0})

lee_merged_filtered['AMPUTATION'] =  (lee_merged_filtered.LEE_MOSTSEVOUTCOME).map({'Death': 0,'Not documented': None,
                                                                             'Patent treated arterial segment, no stenosis':0,
                                                                             'Patent treated arterial segment with stenosis':0,
                                                                             'Reintervened treated arterial segment with stenosis':0,
                                                                             'Major Amputation':1, 'Image-proven treated arterial segment thrombosis or clinically evident thrombosis with no planned intervention':0,
                                                                             'Other':0,'New bypass in the treated arterial segment':0,
                                                                             'Clinically Patent Graft':0,'Reintervened treated arterial segment with no current stenosis':0})

lee_merged_filtered.loc[(lee_merged_filtered.UNPLANREADMISSION == 'Yes') | (lee_merged_filtered.UNPLANNEDREADMISSION1 == 'Yes'), 'unplanned_readmission'] = 1
lee_merged_filtered.loc[(lee_merged_filtered.UNPLANREADMISSION == 'No') & (lee_merged_filtered.UNPLANNEDREADMISSION1 == 'No'), 'readmission'] = 0

lee_merged_filtered.loc[(lee_merged_filtered.REOPERATION == 'Yes') | (lee_merged_filtered.REOPERATION1 == 'Yes'), 'unplanned_reoperation'] = 1
lee_merged_filtered.loc[(lee_merged_filtered.REOPERATION == 'No') & (lee_merged_filtered.REOPERATION1 == 'No'), 'unplanned_reoperation'] = 0

survival_analysis = lee_merged_filtered[['DOPERTOD', 'unplanned_readmission', 'unplanned_reoperation','RETORPODAYS','READMPODAYS1']]
lee_merged_filtered = lee_merged_filtered.drop(columns=['DOPERTOD','UNPLANREADMISSION','UNPLANNEDREADMISSION1',
                                                        'REOPERATION','REOPERATION1',
                                                        'RETORPODAYS','READMPODAYS1','unplanned_readmission', 
                                                        'readmission','unplanned_reoperation',
                                                        'LEE_SYMPT','RACE_NEW','ETHNICITY_HISPANIC',
                                                        'WEIGHT','HEIGHT','DIABETES','SMOKE',
                                                        'DYSPNEA','FNSTATUS2','VENTILAT','HXCOPD',
                                                        'ASCITES','HXCHF','HYPERMED','RENAFAIL',
                                                        'DIALYSIS','WNDINF','STEROID','WTLOSS',
                                                        'BLEEDDIS','TRANSFUS','ASACLAS','WNDCLAS',
                                                        'LEE_MOSTSEVOUTCOME','LEE_MRTAS'])

In [ ]:
# writing the dataset for survival analysis
survival_analysis.to_csv("survival.csv") 

In [ ]:
outcomes = lee_merged_filtered[["LEE_ULP", "LEE_BLEEDING","LEE_MI_STROKE",'SUPINFEC','WNDINFD','ORGSPCSSI',
                               "DEHIS","OUPNEUMO",'REINTUB','PULEMBOL','FAILWEAN','RENAINSF','OPRENAFL',
                               "URNINFEC",'CNSCVA','CDARREST','CDMI','OTHBLEED','OTHDVT','OTHSYSEP',
                               'OTHSESHOCK','THROMBOSIS','AMPUTATION','DEATH']]

lee_merged_filtered.drop(columns=outcomes.columns,inplace=True)
lee_merged_filtered = pd.concat([lee_merged_filtered, outcomes], axis=1)
lee_merged_filtered.drop(columns=['PUFYEAR'],inplace=True)
lee_merged_filtered['AGE'] = lee_merged_filtered['AGE'].replace("90+",90)

# Ready for imputation!
lee_merged_filtered.to_csv("NSQIP_LEE_2008-2018_toimpute.csv",index=False)

In [ ]:
# Imputation

# If running from a new machine, you will have to follow the instructions for the interpretableai package: 
# https://docs.interpretable.ai/stable/OptImpute/

to_impute = pd.read_csv("NSQIP_LEE_2008-2018_toimpute.csv")

train_data = to_impute.head(7429) #2011-2015
df_imputed_train = iai.impute(train_data)

val_data = to_impute[7429:9407] #2016
df_imputed_val = iai.impute(val_data)

test_data = to_impute.tail(5037)
test_data.drop(columns=['PRPT'],inplace=True) #this column is all NAs for the test data so I'm dropping it
df_imputed_test = iai.impute(test_data)

df_imputed_train.to_csv("NSQIP_LEE_2008-2018_optimpute_train_mortality_readmission.csv",index=None)
df_imputed_val.to_csv("NSQIP_LEE_2008-2018_optimpute_val_mortality_readmission.csv",index=None)
df_imputed_test.to_csv("NSQIP_LEE_2008-2018_optimpute_test_mortality_readmission.csv",index=None)

In [ ]:
# Feature selection with mRMR

df_train = pd.read_csv("NSQIP_LEE_2008-2018_optimpute_train_mortality_readmission.csv")
df_val = pd.read_csv("NSQIP_LEE_2008-2018_optimpute_val_mortality_readmission.csv")
df_test = pd.read_csv("NSQIP_LEE_2008-2018_optimpute_test_mortality_readmission.csv")

outcome_colnames = ['LEE_ULP', 'LEE_BLEEDING','LEE_MI_STROKE','SUPINFEC','WNDINFD','ORGSPCSSI',
                    'DEHIS','OUPNEUMO','REINTUB','PULEMBOL','FAILWEAN','RENAINSF','OPRENAFL',
                    'URNINFEC','CNSCVA','CDARREST','CDMI','OTHBLEED','OTHDVT','OTHSYSEP',
                    'OTHSESHOCK','THROMBOSIS','AMPUTATION','DEATH']
outcomes_train = df_train[outcome_colnames]
outcomes_val = df_val[outcome_colnames]
outcomes_test = df_test[outcome_colnames]

predictors_train = df_train.drop(columns=outcome_colnames)
predictors_train = predictors_train.drop(columns=['LEE_AMPUTATION','LEE_WOUND'])

predictors_val = df_val.drop(columns=outcome_colnames)
predictors_val = predictors_val.drop(columns=['LEE_AMPUTATION','LEE_WOUND'])

predictors_test = df_test.drop(columns=outcome_colnames)
predictors_test = predictors_test.drop(columns=['LEE_AMPUTATION','LEE_WOUND'])

outcomes_train.DEATH = outcomes_train.DEATH.round()
outcomes_val.DEATH = outcomes_val.DEATH.round()
outcomes_test.DEATH = outcomes_test.DEATH.round()

outcome_to_test = "DEATH" #CHANGE TO THE OUTCOME YOU WANT TO TEST

X_train = predictors_train
X_val = predictors_val
X_test = predictors_test

y_train = outcomes_train[outcome_to_test]
y_val = outcomes_val[outcome_to_test]
y_test = outcomes_test[outcome_to_test]

In [ ]:
# We want to select approximately the best number of features to start off with 
# (we can narrow it down later by looking at the importance plots and taking out the least important features). 
# We do this by building new models with different numbers of features as selected by mRMR.

def feature_selection_mrmr(feature_name, start_num, end_num, interval_num, X_train, y_train, X_val, y_val):

    selected_features_best = [] 
    top_auc = 0
    
    for i in range(start_num, end_num, interval_num):
        print(i)
        feature = copy.copy(predictors_train)
        feature.insert(loc=0, column=feature_name, value=outcomes_train[feature_name])
        selected_features = pymrmr.mRMR(feature, 'MIQ', i)
        new_predictors = predictors_train[selected_features]

        outcome = feature_name
        oversample = ADASYN()
        X, y = oversample.fit_resample(X_train, y_train)

        clf_rf = RandomForestClassifier(max_depth=4, random_state=0)
        clf_rf.fit(X, y)
        print(outcome)
        print("Accuracy: " + str(clf_rf.score(X_val, y_val)))

        probs = clf_rf.predict_proba(X_val)
        probs = probs[:, 1]
        auc = roc_auc_score(y_val, probs)
        if auc > top_auc:
            top_auc = auc
            selected_features_best = selected_features
        fpr, tpr, thresholds = metrics.roc_curve(y_val, probs)
        
        print("AUC: " + str(auc) + '\n')
        
        pr = average_precision_score(y_val, probs)
        print("Precision: " + str(pr) + '\n')
        
    return selected_features_best

In [ ]:
selected_features = feature_selection_mrmr(outcome_to_test, 1, len(X_train.columns), 5, X_train, y_train, X_val, y_val)

In [ ]:
# View selected features
selected_features